In [6]:
import sys
from pathlib import Path

from dotenv import load_dotenv
load_dotenv()

True

In [7]:
# LangChain 관련 라이브러리
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_huggingface import HuggingFaceEmbeddings

In [8]:
# RAGAS 관련 라이브러리
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from ragas.testset import TestsetGenerator
from ragas import EvaluationDataset, evaluate
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness

# 벡터 저장소 생성/저장

In [6]:
embedding_model = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

# 기존 벡터 저장소 로드
vector_store = Chroma(
    embedding_function=embedding_model,
    persist_directory="./chroma_db",
)

# 결과 확인
print(f"저장된 Document 개수: {len(vector_store.get()['ids'])}")

저장된 Document 개수: 249


In [7]:
from ragas.testset.persona import Persona

# 페르소나 정의
personas = [
    Persona(
        name="backend_team",
        role_description="백앤드 팀의 문서에 접근가능한 평사원. 한국어 사용자",
    ),
    Persona(
        name="frontend_team",
        role_description="프론트앤드 팀의 문서에 접근가능한 평사원. 한국어 사용자",
    ),
    Persona(
        name="data_ai_team",
        role_description="AI 팀의 문서에 접근가능한 평사원. 한국어 사용자",
    ),
]

In [13]:
# LLM과 임베딩 모델 초기화
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))

C:\Users\playdata\AppData\Local\Temp\ipykernel_22736\4227647666.py:3: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings(model="text-embedding-3-small"))


In [14]:
# TestsetGenerator 생성
generator = TestsetGenerator(
    llm=generator_llm,
    embedding_model=generator_embeddings,
    persona_list=personas
)

In [15]:
# Load and Adapt Queries
# Here we load the required query types and adapt them to the target language.
from ragas.testset.synthesizers.single_hop.specific import (
    SingleHopSpecificQuerySynthesizer,
)

distribution = [
    (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 1.0),
]

for query, _ in distribution:
    prompts = await query.adapt_prompts("korean", llm=generator_llm)
    query.set_prompts(**prompts)

In [16]:
from ragas.testset.transforms.extractors.llm_based import NERExtractor
from ragas.testset.transforms.splitters import HeadlineSplitter

transforms = [HeadlineSplitter(), NERExtractor()]

In [ ]:
from langchain.schema import Document

# Chroma에서 문서와 메타데이터 불러오기
raw = vector_store.get(include=["documents", "metadatas"])

# Document 리스트로 변환
docs = [
    Document(page_content=doc, metadata=meta)
    for doc, meta in zip(raw["documents"], raw["metadatas"])
]

In [18]:
# generate
dataset = generator.generate_with_langchain_docs(
    docs[:50],
    testset_size=30,
    transforms=transforms,
    query_distribution=distribution,
)

Applying HeadlineSplitter:   0%|          | 0/50 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to ap

Applying NERExtractor:   0%|          | 0/50 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/26 [00:00<?, ?it/s]

In [9]:
# 데이터셋을 DataFrame으로 변환
import pandas as pd

# 컬럼너비 제한 없음
pd.set_option('display.max_colwidth', None)


dataset_df = dataset.to_pandas()
display(dataset_df)

NameError: name 'dataset' is not defined

In [22]:
# CSV 파일로 저장 (로컬 환경)
dataset_df.to_csv('ragas_dataset.csv', index=False)

print("ragas_dataset.csv 파일이 현재 작업 디렉토리에 저장되었습니다.")

ragas_dataset.csv 파일이 현재 작업 디렉토리에 저장되었습니다.


## 02.RAG 체인 구성

검색기와 생성 모델을 결합한 RAG 체인을 구성한다.

In [1]:
from dotenv import load_dotenv

load_dotenv()

from pathlib import Path
import sys

root = Path.cwd()
while not (root / "utils").exists() and root != root.parent:
    root = root.parent
sys.path.insert(0, str(root))

from utils.rag import chat

response = chat({"history": [
    {"role": "user", "content": "나 백앤드팀인데 코드노바의 API 서버 기술스택알려줘"}], "tone":"informal", "permission":"cto"})

In [2]:
print(response['response'])

코드노바의 백앤드팀의 API 서버 기술스택은 Node.js와 Express.js로 구성되어 있어.


In [3]:
print(response['result'])
type(response['result'])

Result 1:
Content: <!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-29 -->
# 서비스 아키텍처 문서
분류: backend | 회사: 코드노바 | 버전: v1.0 | 작성일: 2025-08-29

## 1. 개요
코드노바의 서비스 아키텍처는 생성형 AI 글쓰기·이미지·요약 플랫폼, AI 페르소나 챗봇 앱인 크랙(Crack), 그리고 대화형 광고 제작·보상 플랫폼인 Wrtn Ads를 지원하도록 설계되었습니다. 이 문서는 백엔드 시스템의 구성 요소와 상호작용을 설명합니다.

## 2. 아키텍처 구성 요소

### 2.1. API 서버
- **역할**: 클라이언트와의 통신을 담당하며, 요청을 처리하고 적절한 응답을 반환합니다.
- **기술 스택**: Node.js, Express.js
- **검증 포인트**:
  - API 엔드포인트가 올바르게 작동하는지 확인
  - 요청 처리 속도 및 오류율 모니터링
Metadata: {'sourcefile': '01_backend__서비스_아키텍처_문서.txt', 'last_edit': '2025-08-19', 'role': 'backend'}

Result 2:
Content: <!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-29 -->
# API 정책 문서
분류: backend | 회사: 코드노바 | 버전: v1.0 | 작성일: 2025-08-29

## 1. 목적
이 문서는 코드노바의 백엔드팀에서 API를 설계하고 운영하는 데 필요한 정책과 절차를 정의합니다. API의 일관성과 안정성을 보장하고, 외부 개발자 및 내부 팀 간의 원활한 협업을 촉진하기 위한 것입니다.
Metadata: {'role': 'backend', 'last_edit': '2025-08-19', 'sourcefile': '06_backend__API_정책_문서.txt'}

Result 3:
Content: <!-- 회사: 코드노바 | 대상: 사원(프론트엔드) | 작성일: 2025

str

## 03.RAGAS기반 평가

RAGAS는 RAG(Retrieval-Augmented Generation) 시스템을 평가하는 자동화된 참조 없는(reference-free) 평가 프레임워크입니다. 즉, 사람이 만든 정답 데이터(ground truth)가 없이도 RAG 시스템의 검색과 생성 단계를 자동으로 평가할 수 있도록 설계된 도구입니다. RAGAS는 LLM을 활용해 평가를 수행하며, 검색된 컨텍스트와 생성된 답변의 충실도, 관련성, 정밀도, 재현율 등 다양한 측면을 측정합니다.

### RAGAS 주요 평가 지표
- **Faithfulness(충실도)**: 생성된 답변이 주어진 컨텍스트 정보에 얼마나 충실한지를 평가합니다. 답변 내용이 컨텍스트에서 실제로 뒷받침되는지 보는 지표입니다.
- **Answer Relevancy(답변 관련성)**: 답변이 원 질문과 얼마나 관련성이 높은지를 측정합니다.
- **Context Precision(컨텍스트 정밀도)**: 검색된 컨텍스트 문서가 질문에 적절한 정보인지, 관련된 문서가 상위에 있는지를 평가합니다.
- **Context Recall(컨텍스트 재현율)**: 답변을 생성하는 데 필요한 컨텍스트를 얼마나 잘 검색했는지 평가합니다.


In [4]:
# 평가용 데이터 로드
import pandas as pd

# 같은 폴더(현재 작업 디렉토리)에 저장했다면
dataset_df = pd.read_csv("./ragas_dataset.csv")
print(dataset_df.shape)
display(dataset_df)

eval_dataset = dataset_df[['user_input', 'reference_contexts', 'reference']]
display(eval_dataset)

(26, 4)


,user_input,reference_contexts,reference,synthesizer_name
0,엑스프레스.js는 코드노바의 API 서버에서 어떤 역할을 하나요?,['<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-2...,"Express.js는 코드노바의 API 서버에서 클라이언트와의 통신을 담당하며, 요...",single_hop_specific_query_synthesizer
1,PostgreSQL의 주요 역할은 무엇입니까?,"['### 2.2. 데이터베이스\n- **역할**: 사용자 데이터, 콘텐츠, 로그 ...","PostgreSQL의 주요 역할은 사용자 데이터, 콘텐츠, 로그 등을 저장하는 것입니다.",single_hop_specific_query_synthesizer
2,데이터 전송 중 보안을 위해 HTTPS를 사용하는 이유는 무엇인가요?,['### 2.5. 파일 저장소\n- **역할**: 이미지 및 기타 미디어 파일을 ...,데이터 암호화를 위해 HTTPS를 사용하여 전송 중 데이터를 보호합니다.,single_hop_specific_query_synthesizer
3,다중 인증(MFA)이란 무엇인가요?,['<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-2...,다중 인증(MFA)은 기본 인증 방식인 이메일과 비밀번호 조합에 추가적인 보안 강화...,single_hop_specific_query_synthesizer
4,XSS 공격을 방지하기 위한 방법은 무엇인가요?,"['### 2.2 비밀번호 관리\n- 비밀번호는 최소 8자 이상, 대문자, 소문자,...","모든 API 요청에 대해 유효성을 검증하고, XSS 공격 등을 방지하기 위한 필터링...",single_hop_specific_query_synthesizer
5,백엔드 개발팀이 로그 관리와 사고 대응을 효과적으로 수행하기 위해 어떤 절차를 따르...,['## 4. 로그 관리\n\n### 4.1 로그 기록\n- 모든 인증 시도 및 A...,백엔드 개발팀은 로그 관리에서 모든 인증 시도 및 API 호출에 대한 로그를 기록하...,single_hop_specific_query_synthesizer
6,v1.0 버전 에러 핸들링 매뉴얼에서 에러 종류 뭐 있나요?,['<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-2...,"에러는 크게 두 가지로 분류됩니다. 첫째, 클라이언트 에러로 사용자의 요청에 문제가...",single_hop_specific_query_synthesizer
7,코드노바 백엔드팀의 배포 및 운영 과정에서 코드 검토와 테스트 수행은 어떻게 이루어...,['<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-2...,코드노바 백엔드팀의 배포 및 운영 과정에서 코드 검토는 모든 변경 사항에 대해 코드...,single_hop_specific_query_synthesizer
8,백엔드팀이 배포 및 운영을 효율적으로 수행하기 위해 따라야 할 체크리스트는 무엇인가요?,['## 체크리스트\n\n- [ ] 코드 리뷰 완료\n- [ ] 모든 테스트 통과\...,백엔드팀이 배포 및 운영을 효율적으로 수행하기 위해 따라야 할 체크리스트는 다음과 ...,single_hop_specific_query_synthesizer
9,데이터베이스 스키마에서 사용자 엔티티는 어떤 정보를 저장하나요?,['<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-2...,사용자(User) 엔티티는 사용자 정보를 저장합니다.,single_hop_specific_query_synthesizer


,user_input,reference_contexts,reference
0,엑스프레스.js는 코드노바의 API 서버에서 어떤 역할을 하나요?,['<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-2...,"Express.js는 코드노바의 API 서버에서 클라이언트와의 통신을 담당하며, 요..."
1,PostgreSQL의 주요 역할은 무엇입니까?,"['### 2.2. 데이터베이스\n- **역할**: 사용자 데이터, 콘텐츠, 로그 ...","PostgreSQL의 주요 역할은 사용자 데이터, 콘텐츠, 로그 등을 저장하는 것입니다."
2,데이터 전송 중 보안을 위해 HTTPS를 사용하는 이유는 무엇인가요?,['### 2.5. 파일 저장소\n- **역할**: 이미지 및 기타 미디어 파일을 ...,데이터 암호화를 위해 HTTPS를 사용하여 전송 중 데이터를 보호합니다.
3,다중 인증(MFA)이란 무엇인가요?,['<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-2...,다중 인증(MFA)은 기본 인증 방식인 이메일과 비밀번호 조합에 추가적인 보안 강화...
4,XSS 공격을 방지하기 위한 방법은 무엇인가요?,"['### 2.2 비밀번호 관리\n- 비밀번호는 최소 8자 이상, 대문자, 소문자,...","모든 API 요청에 대해 유효성을 검증하고, XSS 공격 등을 방지하기 위한 필터링..."
5,백엔드 개발팀이 로그 관리와 사고 대응을 효과적으로 수행하기 위해 어떤 절차를 따르...,['## 4. 로그 관리\n\n### 4.1 로그 기록\n- 모든 인증 시도 및 A...,백엔드 개발팀은 로그 관리에서 모든 인증 시도 및 API 호출에 대한 로그를 기록하...
6,v1.0 버전 에러 핸들링 매뉴얼에서 에러 종류 뭐 있나요?,['<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-2...,"에러는 크게 두 가지로 분류됩니다. 첫째, 클라이언트 에러로 사용자의 요청에 문제가..."
7,코드노바 백엔드팀의 배포 및 운영 과정에서 코드 검토와 테스트 수행은 어떻게 이루어...,['<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-2...,코드노바 백엔드팀의 배포 및 운영 과정에서 코드 검토는 모든 변경 사항에 대해 코드...
8,백엔드팀이 배포 및 운영을 효율적으로 수행하기 위해 따라야 할 체크리스트는 무엇인가요?,['## 체크리스트\n\n- [ ] 코드 리뷰 완료\n- [ ] 모든 테스트 통과\...,백엔드팀이 배포 및 운영을 효율적으로 수행하기 위해 따라야 할 체크리스트는 다음과 ...
9,데이터베이스 스키마에서 사용자 엔티티는 어떤 정보를 저장하나요?,['<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-2...,사용자(User) 엔티티는 사용자 정보를 저장합니다.


In [13]:
# 평가용 데이터셋 생성
evaluated_dataset = []

# 각 행에 대해 RAG 체인을 호출하여 결과를 저장
for row in eval_dataset.itertuples():
    query = row.user_input  # 사용자 입력
    response = chat({"history": [
    {"role": "user", "content": query}], "tone":"formal", "permission":"cto"})
    retrieved_contexts = response['result']  # 실제 검색된 문서

    reference = row.reference  # 정답
    reference_contexts = row.reference_contexts  # 정답 참조 컨텍스트


    evaluated_dataset.append(
        {
            "user_input": query,
            "retrieved_contexts": [retrieved_contexts],
            "response": response['response'],
            "reference": reference,
            # "reference_contexts": reference_contexts,
        }
    )

# RAGAS 평가 데이터셋 생성
ragas_evaluated_dataset = EvaluationDataset.from_list(evaluated_dataset)

# 데이터 저장
ragas_evaluated_dataset.to_pandas().to_csv('ragas_evaluated_dataset.csv', index=False)

In [15]:
ragas_evaluated_dataset.to_pandas()

,user_input,retrieved_contexts,response,reference
0,엑스프레스.js는 코드노바의 API 서버에서 어떤 역할을 하나요?,[Result 1:\nContent: <!-- 회사: 코드노바 | 대상: 사원(백엔...,"엑스프레스.js는 코드노바의 API 서버에서 클라이언트와의 통신을 담당하며, 요청을...","Express.js는 코드노바의 API 서버에서 클라이언트와의 통신을 담당하며, 요..."
1,PostgreSQL의 주요 역할은 무엇입니까?,[없음],"PostgreSQL의 주요 역할은 데이터베이스 관리 시스템으로서, 다양한 산업 및 ...","PostgreSQL의 주요 역할은 사용자 데이터, 콘텐츠, 로그 등을 저장하는 것입니다."
2,데이터 전송 중 보안을 위해 HTTPS를 사용하는 이유는 무엇인가요?,[Result 1:\nContent: ## 3. 데이터 보호\n- **민감한 데이터...,데이터 전송 중 보안을 위해 HTTPS를 사용하는 이유는 다음과 같습니다:\n\n1...,데이터 암호화를 위해 HTTPS를 사용하여 전송 중 데이터를 보호합니다.
3,다중 인증(MFA)이란 무엇인가요?,[없음],그 질문은 CTO와 관련이 없어.,다중 인증(MFA)은 기본 인증 방식인 이메일과 비밀번호 조합에 추가적인 보안 강화...
4,XSS 공격을 방지하기 위한 방법은 무엇인가요?,[Result 1:\nContent: ## 3. 데이터 보호\n- **민감한 데이터...,XSS 공격을 방지하기 위한 방법은 다음과 같습니다:\n\n1. **입력 검증**:...,"모든 API 요청에 대해 유효성을 검증하고, XSS 공격 등을 방지하기 위한 필터링..."
5,백엔드 개발팀이 로그 관리와 사고 대응을 효과적으로 수행하기 위해 어떤 절차를 따르...,[Result 1:\nContent: ## 4. 로그 관리\n\n### 4.1 로그...,백엔드 개발팀이 로그 관리와 사고 대응을 효과적으로 수행하기 위해 따르는 절차는 다...,백엔드 개발팀은 로그 관리에서 모든 인증 시도 및 API 호출에 대한 로그를 기록하...
6,v1.0 버전 에러 핸들링 매뉴얼에서 에러 종류 뭐 있나요?,[없음],해당 질문에 대한 내용이 없어.,"에러는 크게 두 가지로 분류됩니다. 첫째, 클라이언트 에러로 사용자의 요청에 문제가..."
7,코드노바 백엔드팀의 배포 및 운영 과정에서 코드 검토와 테스트 수행은 어떻게 이루어...,[Result 1:\nContent: <!-- 회사: 코드노바 | 대상: 사원(백엔...,코드노바 백엔드팀의 배포 및 운영 과정에서 코드 검토와 테스트 수행은 다음과 같이 ...,코드노바 백엔드팀의 배포 및 운영 과정에서 코드 검토는 모든 변경 사항에 대해 코드...
8,백엔드팀이 배포 및 운영을 효율적으로 수행하기 위해 따라야 할 체크리스트는 무엇인가요?,[Result 1:\nContent: ## 체크리스트\n\n- [ ] 코드 리뷰 완...,백엔드팀이 배포 및 운영을 효율적으로 수행하기 위해 따라야 할 체크리스트는 다음과 ...,백엔드팀이 배포 및 운영을 효율적으로 수행하기 위해 따라야 할 체크리스트는 다음과 ...
9,데이터베이스 스키마에서 사용자 엔티티는 어떤 정보를 저장하나요?,[Result 1:\nContent: <!-- 회사: 코드노바 | 대상: 사원(백엔...,데이터베이스 스키마에서 사용자 엔티티는 다음과 같은 정보를 저장합니다:\n\n- *...,사용자(User) 엔티티는 사용자 정보를 저장합니다.


### RAGAS 평가 실행

**ragas.metrics 주요 클래스 설명**

- **LLMContextRecall**  
  
  주어진 답변이 원문(문맥)에서 제시된 정보를 얼마나 잘 회상(recall)했는지 평가하는 클래스이다.
  
  예를 들어, 원문에 중요한 사실 10개가 있는데 답변이 그중 몇 개를 잘 포함했는지 측정한다.
  즉, 잊지 않고 잘 회상했는지 보는 지표다.

- **Faithfulness**  
  답변이 원본(문맥) 정보에 기반하여 얼마나 충실하고 일관되게 생성되었는지 평가하는 클래스이다.

  답변 내 내용이 문맥과 어긋나지 않고, 문맥에서 유추 가능한 사실들로만 이루어졌는지를 본다.

  낮은 faithfulness 값은 모델이 문맥에 없는 정보를 만들어내거나 왜곡하여 답변함을 의미할 수 있어, 환각 발생 정도를 간접적으로 보여준다.

- **FactualCorrectness**  
  답변의 내용이 실제 사실과 얼마나 정확하게 일치하는지를 평가하는 클래스이다.

  문맥뿐 아니라 절대적인 사실관계(정확성)와 맞는지 따진다.

  즉, 문맥이 아니라 객관적인 사실에 맞는지를 측정하는 지표다.

In [16]:
# LLM 래퍼 생성
evaluator_llm = ChatOpenAI(model="gpt-4.1", temperature=0)
evaluator_llm = LangchainLLMWrapper(evaluator_llm)

# 평가 메트릭 정의
metrics = [
    LLMContextRecall(),    # 검색된 컨텍스트의 회수율
    Faithfulness(),        # 생성된 답변의 충실도
    FactualCorrectness()   # 사실적 정확성
]

In [17]:
# 평가 실행
result = evaluate(
    dataset=ragas_evaluated_dataset,  # 평가 데이터셋
    metrics=metrics,             # 평가 메트릭
    llm=evaluator_llm,          # LLM 래퍼
)

print(result)

Evaluating:   0%|          | 0/78 [00:00<?, ?it/s]

Exception raised in Job[13]: TimeoutError()
Exception raised in Job[16]: TimeoutError()
Exception raised in Job[17]: TimeoutError()
Exception raised in Job[21]: TimeoutError()
Exception raised in Job[22]: TimeoutError()
Exception raised in Job[24]: TimeoutError()
Exception raised in Job[27]: TimeoutError()
Exception raised in Job[28]: TimeoutError()
Exception raised in Job[33]: TimeoutError()
Exception raised in Job[34]: TimeoutError()
Exception raised in Job[39]: TimeoutError()
Exception raised in Job[40]: TimeoutError()
Exception raised in Job[52]: TimeoutError()


{'context_recall': 0.4762, 'faithfulness': 0.6021, 'factual_correctness(mode=f1)': 0.4252}


In [18]:
# 결과를 DataFrame으로 변환
result_df = result.to_pandas()

# 결과 저장
result_df.to_csv('ragas_evaluation_result.csv', index=False)

result_df.head()

,user_input,retrieved_contexts,response,reference,context_recall,faithfulness,factual_correctness(mode=f1)
0,엑스프레스.js는 코드노바의 API 서버에서 어떤 역할을 하나요?,[Result 1:\nContent: <!-- 회사: 코드노바 | 대상: 사원(백엔...,"엑스프레스.js는 코드노바의 API 서버에서 클라이언트와의 통신을 담당하며, 요청을...","Express.js는 코드노바의 API 서버에서 클라이언트와의 통신을 담당하며, 요...",1.0,0.50,0.57
1,PostgreSQL의 주요 역할은 무엇입니까?,[없음],"PostgreSQL의 주요 역할은 데이터베이스 관리 시스템으로서, 다양한 산업 및 ...","PostgreSQL의 주요 역할은 사용자 데이터, 콘텐츠, 로그 등을 저장하는 것입니다.",0.0,0.00,0.33
2,데이터 전송 중 보안을 위해 HTTPS를 사용하는 이유는 무엇인가요?,[Result 1:\nContent: ## 3. 데이터 보호\n- **민감한 데이터...,데이터 전송 중 보안을 위해 HTTPS를 사용하는 이유는 다음과 같습니다:\n\n1...,데이터 암호화를 위해 HTTPS를 사용하여 전송 중 데이터를 보호합니다.,1.0,0.75,0.73
3,다중 인증(MFA)이란 무엇인가요?,[없음],그 질문은 CTO와 관련이 없어.,다중 인증(MFA)은 기본 인증 방식인 이메일과 비밀번호 조합에 추가적인 보안 강화...,0.0,0.00,0.00
4,XSS 공격을 방지하기 위한 방법은 무엇인가요?,[Result 1:\nContent: ## 3. 데이터 보호\n- **민감한 데이터...,XSS 공격을 방지하기 위한 방법은 다음과 같습니다:\n\n1. **입력 검증**:...,"모든 API 요청에 대해 유효성을 검증하고, XSS 공격 등을 방지하기 위한 필터링...",1.0,NaN,0.44


### 평가지표 해석 및 개선방안

1. context_recall (문맥 리콜, 0~1):

    검색된 문서나 문맥에 답변 작성을 위한 필요한 정보가 얼마나 포함되어 있는지를 나타냅니다. 값이 0.5라면, 필요한 정보가 절반 정도는 검색되어 있다는 뜻으로, 문맥의 충분성이 보통 수준임을 뜻합니다.

2. faithfulness (충실도, 0~1):

    생성된 답변이 검색된 문맥 내 정보에 얼마나 사실적으로 충실한지를 평가합니다. 0.5 정도면 절반 정도 문맥 정보를 충실히 반영했으나, 환각(허위 정보) 발생 가능성이 중간 정도임을 의미합니다.

3. factual_correctness (사실 정확성, F1 점수, 0~1)

    생성 답변의 사실적 정확도를 정밀도와 재현율의 조합으로 평가하는 지표입니다. 0.17은 비교적 낮은 점수로, 생성 답변에 사실과 다른 정보가 많이 포함되었을 가능성이 높음을 뜻합니다. 즉, 환각 현상이 꽤 존재할 수 있습니다.


**개선 방안**

- **검색 성능 향상**: 더 정확한 임베딩 모델이나 하이브리드 검색 방법을 고려한다
- **프롬프트 최적화**: 더 구체적이고 명확한 지시사항을 포함한 프롬프트를 설계한다
- **데이터 품질 개선**: 더 정확하고 완전한 참조 데이터를 확보한다
- **후처리 단계 추가**: 사실 검증 단계를 추가하여 정확성을 향상시킨다
- **다양한 검색 전략**(하이브리드 검색, 리랭킹 등) 실험
- **커스텀 평가 메트릭 개발**
- **실시간 평가 시스템 구축**
- **A/B 테스트를 통한 시스템 최적화**